In [11]:
!pip install pulp

from pulp import *

In [7]:
import numpy as np
import pandas as pd


# Warehouse Network Optimization

You are working on a distribution plan for a warehouse network. The network has two warehouses (W1, and W2) and each can ship three different types of products (A, B, and C). W1 is small and can either ship 10 products A per a week or 15 products B per a week or 20 products C per a week. 

> ## You are looking to minimize the total costs.

In [5]:
cust = ['C1','C2','C3','C4']

wh = ['W1','W2']

prod = ['A', 'B', 'C']

In [8]:
demand = pd.DataFrame({'C1':[10,17,23], 'C2':[8,11,20],'C3':[28,10,15],'C4':[0,6,13]}, 
                      index=['A','B','C'])

demand

,C1,C2,C3,C4
A,10,8,28,0
B,17,11,10,6
C,23,20,15,13


In [24]:
costs = pd.DataFrame({('W1','A'):[81,92,110,130],('W1','B'):[117,77,108,98],('W1','C'):[107,97,102,104],
             ('W2','A'):[102,105,115,130],('W2','B'):[189,132,200,192],('W2','C'):[108,98,103,105]}, 
            index=cust).T

# costs.loc[('W1','A'),'C1']

costs

C1   C2   C3   C4
W1 A   81   92  110  130
   B  117   77  108   98
   C  107   97  102  104
W2 A  102  105  115  130
   B  189  132  200  192
   C  108   98  103  105

# Initialize, Define Decision Vars., Objective Function, and Constraints

In [14]:
model = LpProblem("Distribution Planning", LpMinimize)

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [15]:
X = LpVariable.dicts("ship", [(w, p, c) for c in cust for p in prod for w in wh], 
                     lowBound=0, cat="Integer")

# Objective Function

In [16]:
model += lpSum([X[(w, p, c)]*costs.loc[(w, p), c]  for c in cust for p in prod for w in wh])

# Constraints

1. Shipment limitation based constraints
2. Supply Demand balance based constraints.

In [17]:
for c in cust:
    for p in prod:
        model += lpSum([X[(w, p, c)] for w in wh]) == demand.loc[p, c]

# Define Dependent Demand Constraints
model += ((1/10) * lpSum([X[('W1', 'A', c)] for c in cust]) 
        + (1/15) * lpSum([X[('W1', 'B', c)] for c in cust])
        + (1/20) * lpSum([X[('W1', 'C', c)] for c in cust]))<=4

# Solve the Model to obtain the respective delivery values

In [18]:
model.solve()

1

## Generate Model Results Report. 

>The result is basically the Number of each product shiped from a warehouse to a customer location.

In [20]:
solutions = [{(w,p,c) : X[(w, p, c)].varValue} for c in cust for p in prod for w in wh]

In [27]:
solns = costs.copy()
solns.values[:,:] = 0

solns

C1  C2  C3  C4
W1 A   0   0   0   0
   B   0   0   0   0
   C   0   0   0   0
W2 A   0   0   0   0
   B   0   0   0   0
   C   0   0   0   0

In [29]:
for w in wh:
  for p in prod:
    for c in cust:

      solns.loc[(w,p),c] = X[(w, p, c)].varValue

In [30]:
solns

C1  C2  C3  C4
W1 A  10   0   0   0
   B  17  11  10   6
   C   0   0   0   1
W2 A   0   8  28   0
   B   0   0   0   0
   C  23  20  15  12

### Conclusion : So, on a high level, for both W1 and W2, more shipments are to places (customers) where the costs of shipments are less relatively. 

Also, based on the constraint, the model has suggested lesser shipments for W1 as compared to W2